## Libraries

In [ ]:
import pandas as pd
import numpy as np
import urllib2
import urllib
import BeautifulSoup
import cookielib
import requests

## DataFrame

In [ ]:
new_jersy_data = pd.DataFrame(columns=['Name', 'Overview', 'Years in practice', 'Language',
       'Office location', 'Hospital Affiliation',
       'Specialties & sub specialties', 'Education & medical training',
       'Certification & licensure'])

## Getting Html Content using BeautifulSoup

In [ ]:
def link(link):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:60.0) Gecko/20100101 Firefox/60.0'}
    html_content = requests.get(link, headers=headers)
    html_content.status_code
    d = (html_content.content)
    return BeautifulSoup.BeautifulSoup(d)

## Function For getting Doctors Information.

In [1]:
def doc_info(soup_4):
    each_doc_info = []
    # Doctor name
    each_doc_info.append(str(soup_4.find('h1',{'class':"hero-heading flex-media-heading block-tight doctor-name "}).getText()))

    # Doctor overview
    each_doc_info.append(str(soup_4.find('div',{'class':"block-normal clearfix"}).getText()))

    # Experience
    exp = soup_4.findAll('span',{'class':"text-large heading-normal-for-small-only right-for-medium-up"})[1].getText()
    #each_doc_info.append(int(''.join(x for x in exp if x.isdigit())))
    each_doc_info.append(exp)
    # Language
    each_doc_info.append(str(soup_4.findAll('span',
                        {'class':"text-large heading-normal-for-small-only right-for-medium-up text-right showmore"})[0].getText()))

    #office_location
    each_doc_info.append(str(soup_4.findAll('span',{'class':"text-strong"})[0].getText()))
    
    hospital_affiliation = soup_4.findAll("div",{"class":"flex-small-12 padding-flush"})
    if hospital_affiliation == []:
        hos_affiliation = "No"
    else:
        a = []
        for i in hospital_affiliation:
            a.append(i.find('a',{'class':'heading-larger block-tight'}).getText())
        hos_affiliation = str(",".join(a))
    each_doc_info.append(hos_affiliation)
    
    # Specialties and sub-specialties
    specialities_sub= soup_4.findAll('div',{'class':"flex-medium-6 flex-small-12"})
    specialties = str(specialities_sub[0].find('a',{'class':'text-large'}).getText())
    sub_specialties = str(specialities_sub[0].find('p',{'class':'text-large block-tight'}).getText())
    special = "Specialties: " +specialties + ", sub-specialties: " + sub_specialties
    each_doc_info.append(special)
    
    # Education
    section = soup_4.findAll('section',{'class':"block-loosest"})
    index_education_section = 0
    for i in range(len(section)):
        if str(section[i].find('h2',{'class':"heading-larger block-normal"}).getText()) == 'Education &amp; Medical Training':
            index_education_section = i
            break
    education_list = section[index_education_section].findAll(['li'])
    education = []
    for i in education_list:
         education.append(" ".join(i.getText().split()))
    education = str(" ".join(education))
    each_doc_info.append(education)
    
    # Certification
    index_certification_section = 0
    for i in range(len(section)):
        if str(section[i].find('h2',{'class':"heading-larger block-normal"}).getText()) == 'Certifications &amp; Licensure':
            index_certification_section = i
            break
    certification = section[index_certification_section].findAll('li')
    certifications = []
    for i in certification:
        certifications.append(str(i.getText()))
    each_doc_info.append(" ".join(certifications))
    return each_doc_info
    
    

## Merging in dataFrame

In [ ]:
def appending_data(info_list):
    j = 0
    row_num = new_jersy_data.shape[0]+1
    for i in new_jersy_data.columns:
        new_jersy_data.loc[row_num,i] = info_list[j]
        j = j+1